In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np


In [8]:
# Define transforms for data augmentation and normalization
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

# Load CIFAR-10 training and test datasets
trainset = torchvision.datasets.CIFAR10(root='C:\\Users\\Conno\\Dropbox\\data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='C:\\Users\\Conno\\Dropbox\\data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


Files already downloaded and verified
Files already downloaded and verified


In [9]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 3, padding=1)
        self.conv2 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv3 = nn.Conv2d(128, 256, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(256 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = self.pool(nn.functional.relu(self.conv3(x)))
        x = x.view(-1, 256 * 4 * 4)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Instantiate the model and move it to GPU if available
net = CNN()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net.to(device)


CNN(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=4096, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=10, bias=True)
)

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)


In [11]:
from sklearn.metrics import accuracy_score, f1_score

for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    correct = 0
    total = 0
    y_true = []
    y_pred = []

    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to GPU if available

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
        # Calculate accuracy and F1 score
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())
    
    epoch_accuracy = correct / total
    epoch_f1_score = f1_score(y_true, y_pred, average='weighted')
    
    print(f'Epoch {epoch + 1}, Loss: {running_loss / len(trainloader)}, Accuracy: {epoch_accuracy}, F1 Score: {epoch_f1_score}')

print('Finished Training')


Epoch 1, Loss: 2.053362442451033, Accuracy: 0.2481, F1 Score: 0.22779276758489037
Epoch 2, Loss: 1.6743059765430726, Accuracy: 0.3866, F1 Score: 0.3781317968371292
Epoch 3, Loss: 1.5008323727245905, Accuracy: 0.45362, F1 Score: 0.4485147855892164
Epoch 4, Loss: 1.3956874886996, Accuracy: 0.49432, F1 Score: 0.489715704626969
Epoch 5, Loss: 1.2852381579554089, Accuracy: 0.54074, F1 Score: 0.5369080645799305
Epoch 6, Loss: 1.1971916235644926, Accuracy: 0.56898, F1 Score: 0.5662079274458414
Epoch 7, Loss: 1.1192771856478216, Accuracy: 0.60258, F1 Score: 0.6002603609034448
Epoch 8, Loss: 1.0578614849351724, Accuracy: 0.62296, F1 Score: 0.6208914907696478
Epoch 9, Loss: 0.9993471240699863, Accuracy: 0.64542, F1 Score: 0.6438737697498578
Epoch 10, Loss: 0.9482312561301794, Accuracy: 0.66846, F1 Score: 0.6672249758698354
Finished Training


In [12]:
# Specify the directory where you want to save the trained model in Dropbox
SAVE_DIR = 'C:\\Users\\Conno\\Dropbox\\'

# Define the file name for the saved model
MODEL_NAME = 'cifar_net.pth'

# Construct the full path
PATH = SAVE_DIR + MODEL_NAME

# Save the trained model
torch.save(net.state_dict(), PATH)
print('Saved trained model at:', PATH)


Saved trained model at: C:\Users\Conno\Dropbox\cifar_net.pth
